## Migration Testing - Leonardo

#### Read in Currents feed

In [1]:
from dotenv import load_dotenv
from currentsapi import CurrentsAPI
import os
import requests
import time
import openai
import random

In [2]:
load_dotenv()
currents_api_key = os.environ['currents_api_key']
leonardo_api_key = os.environ['leonardo_key']
openai.api_key = os.environ['openai_key']

In [3]:
api = CurrentsAPI(api_key=currents_api_key)

In [4]:
news_feed = api.search(country="", language="en", category="world")

#### Process into GPT prompt

In [5]:
titles = [x['title'] for x in news_feed['news']]

In [6]:
selected_snippet = random.choice(titles)

In [7]:
selected_snippet

'Three Far-Right Groups Face Ban in France - News18'

In [8]:
prompt_template = '''
Using the following snippet encased in ```, generate an image prompt of an imaginary character that is relevant to the snippet, be extravagent and detailed in the description of this character including the type of creature (if applicable), race, age group, ethnicity and appearance. Ensure that the overall character design aligns with the original snippet and a random action pose is included in the description.

Also come up with a creative backstory in around 50 words on this character's origin story and his/her/its main superpower, and include elements of the original snippet in the backstory.

```Russia Extends Detention of US Journalist```
'''  # noqa: E501

response_template = '''
Imaginary Character: Lumis Shadowcloak, Guardian of Free Thought

Description:
Lumis, an enigmatic ethereal being with flowing robes that morph into celestial constellations. Their eyes gleam with cosmic wisdom, reflecting the ages. Lumis stands defiant, one hand reaching for the stars, the other shielding the truth-seekers from the oppressive shadows.

Backstory:
Lumis, born from the celestial whispers of suppressed voices, is the embodiment of free thought. Originating in the astral plane, they journey through dimensions to champion journalists. Their power, Celestial Ward, shields truth-seekers against oppressive forces.
'''

In [9]:
completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                'role': 'system',
                'content': 'You are a creative character designer'
            },
            {
                'role': 'user',
                'content': prompt_template
            },
            {
                'role': 'assistant',
                'content': response_template
            },
            {
                'role': 'user',
                'content': f'Do the same for the following snippet: `{selected_snippet}`'  # noqa: E501
            }
        ]
    )

image_prompt = completion.choices[0].message.content

print(selected_snippet)
print(image_prompt)

Three Far-Right Groups Face Ban in France - News18
Imaginary Character: Valeria Stormrider, Bastion of Equality

Description:
Valeria, a fierce warrior with flowing silver hair and piercing blue eyes, dons an intricate armor adorned with symbols of unity. Standing tall with a confident expression, she wields a gleaming silver staff, channeling her potent power. Her aura radiates strength and determination, instilling hope in the hearts of those fighting for equality.

Backstory:
Valeria, a descendant of ancient druids, possesses an innate connection to the elements. Raised in the lush forests of France, she witnessed the rise of extremist groups threatening harmony. Gifted with the power of Elemental Harmony, she harnesses the forces of nature to combat hatred and division, and strives to bring balance back to society.


In [10]:
prompt = image_prompt.replace('\n', '').split('Description:')[-1].split('Backstory:')[0]
post_text = image_prompt.replace('\n', '').split('Backstory:')[-1]

In [11]:
p1 = post_text.split('Main Superpower:')[0]
p2 = post_text.split('Main Superpower:')[-1].split(' - ')[-1]
final_text = p1 + ' ' + p2

In [12]:

elements_endpoint = "https://cloud.leonardo.ai/api/rest/v1/elements"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

elements_response = requests.get(elements_endpoint, headers=headers)

In [24]:
[x['baseModel'] for x in elements_response.json()['loras']]

['v1_5',
 'SDXL_1_0',
 'v1_5',
 'v1_5',
 'SDXL_1_0',
 'v1_5',
 'v1_5',
 'v1_5',
 'v1_5',
 'SDXL_1_0',
 'v1_5',
 'v1_5',
 'v1_5',
 'SDXL_1_0',
 'v1_5',
 'v2',
 'v1_5',
 'v1_5']

In [13]:
elem_list = [(x['akUUID'], x['weightMax']) for x in elements_response.json()['loras']]
selected_elem = random.choice(elem_list)

#### Call Leonardo AI

In [20]:

url = "https://cloud.leonardo.ai/api/rest/v1/generations"

payload = {
    "modelId": "1e60896f-3c26-4296-8ecc-53e2afecc132",
    "prompt": prompt,
    "height": 512,
    "width": 512,
    "alchemy": True,
    "highResolution": True,
    "elements": [
        {
            "akUUID": selected_elem[0],
            "weight": selected_elem[-1]
        }
    ],
    "highResolution": True,
    "nsfw": True,
    "num_images": 1,
    #"photoReal": True,
    #"photoRealStrength": 0.45,
    "presetStyle": "CINEMATIC",
    "public": True,
    "expandedDomain": True
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)
for k in range(20):
    print(f'Sleeping...{k}')
    time.sleep(1)

{"error":"elements input is not compatible with sdVersion: SDXL_0_9","path":"$","code":"unexpected"}
Sleeping...0
Sleeping...1
Sleeping...2
Sleeping...3
Sleeping...4
Sleeping...5
Sleeping...6
Sleeping...7
Sleeping...8
Sleeping...9
Sleeping...10
Sleeping...11
Sleeping...12
Sleeping...13
Sleeping...14
Sleeping...15
Sleeping...16
Sleeping...17
Sleeping...18
Sleeping...19


In [21]:
response.json()

{'error': 'elements input is not compatible with sdVersion: SDXL_0_9',
 'path': '$',
 'code': 'unexpected'}

In [22]:
gen_id = response.json()['sdGenerationJob']['generationId']

KeyError: 'sdGenerationJob'

#### Grab the generated image from a CDN

In [ ]:
url = f"https://cloud.leonardo.ai/api/rest/v1/generations/{gen_id}"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"generations_by_pk":{"generated_images":[{"url":"https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/b46280d5-b877-4e49-8c23-556e6abce5ed/Leonardo_Diffusion_XL_Aurora_Frostwalker_a_majestic_elemental_0.jpg","nsfw":true,"id":"be445f63-980f-4a06-9f89-df77478ddad9","likeCount":0,"generated_image_variation_generics":[]}],"modelId":"1e60896f-3c26-4296-8ecc-53e2afecc132","prompt":"Aurora Frostwalker, a majestic elemental with a slender, ethereal body encased in shimmering ice armor. Crystal-blue eyes, mirroring the frozen tundra, hold a mixture of compassion and determination. Long, flowing hair made of frost, dances in the frigid winds. Aurora stands tall, her outstretched arms emanating an icy aura that envelops the rescued souls, protecting them from the harsh snowstorm.","negativePrompt":"","imageHeight":512,"imageWidth":512,"inferenceSteps":30,"seed":282296576,"public":false,"scheduler":"EULER_DISCRETE","sdVersion":"SDXL_0_9","status":"COMPLETE","presetStyle"

In [ ]:
print(final_text)

Born from the sacred glaciers, Aurora Frostwalker became the guardian of the frozen lands. As her powers awakened, she felt an empathic connection with those in need. She traverses the treacherous terrains, using her Frost Serenade ability to summon blizzards to rescue stranded individuals, safeguarding them from snowstorms and guiding them to safety. Aurora's heart is warmed by their grateful smiles. Born from the sacred glaciers, Aurora Frostwalker became the guardian of the frozen lands. As her powers awakened, she felt an empathic connection with those in need. She traverses the treacherous terrains, using her Frost Serenade ability to summon blizzards to rescue stranded individuals, safeguarding them from snowstorms and guiding them to safety. Aurora's heart is warmed by their grateful smiles.
